In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, utils
from cifar import CIFAR10
from pathlib import Path


In [3]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)

resnext50_32x4d = models.resnext50_32x4d(pretrained=True)


# Do not forget to enable GPUs!

In [4]:
def use_gpu():
    torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available() 
                                                         else torch.FloatTensor)
use_gpu()

# You might want to use custom transforms

In [5]:
transform = transforms.Compose([transforms.ToTensor()])

# Download the dataset

In [6]:

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=False, transform=transform)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=False, transform=transform)
trainset = CIFAR10(root='./data', train=True,download=True, transform=transform)

testset = CIFAR10(root='./data', train=True,download=True, transform=transform)
print(trainset.targets)
# classess = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
classess = ('Positive', 'Negative')

hello
Files already downloaded and verified
start
./data
Data
one.pickle
stop
/Users/games/Desktop/ML/Data/one.pickle
hello
start
./data
Data
two.pickle
stop
/Users/games/Desktop/ML/Data/two.pickle
hello
start
./data
Data
three.pickle
stop
/Users/games/Desktop/ML/Data/three.pickle
hello
start
./data
Data
four.pickle
stop
/Users/games/Desktop/ML/Data/four.pickle
hello
start
./data
Data
five.pickle
stop
/Users/games/Desktop/ML/Data/five.pickle
hello
5
wassupsss
22
110
wassupsss
(110, 3, 32, 32) hello
110
wassupsss
(110, 3, 32, 32) hellos
110
[[[[ 13  13  13]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[ 11  11  11]
   [  0   0   0]
   [  3   3   3]
   ...
   [ 11  11  11]
   [  1   1   1]
   [  0   0   0]]

  [[ 44  44  44]
   [ 27  27  27]
   [ 49  49  49]
   ...
   [ 60  60  60]
   [ 38  38  38]
   [ 19  19  19]]

  ...

  [[  3   3   3]
   [  0   0   0]
   [ 46  46  46]
   ...
   [ 89  89  89]
   [ 47  47  47]
   [  0   0   0]]

  [

In [7]:
print(len(trainset))
print(len(testset))


110
110


In [25]:


def train(model, train_loader, optimizer, epoch):
    model.train()
    # problem in the following for loop
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {}, Loss: {:.3f}'.format(epoch, loss.item()))


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            print(data)
            output = model(data)
            _, pred = output.max(1)
            print(pred[0].item())
            test_loss += criterion(output, target).item()

            correct += pred.eq(target).sum().item()
            total += target.size(0)
    test_loss /= len(test_loader.dataset)

    print('Average loss: {:.3f}, Test Acc: {:.3f} ({}/{})'.format(test_loss, 100.*correct/total, correct, total))
    
    return test_loss


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(4,4), padding=1), #used to be 4 by 4
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(2048, 1024), #used to be 18432, 3072
            nn.ReLU(),
            nn.Linear(1024, 512), #used to be 3072, 1024
            nn.ReLU(),
            nn.Linear(512, 256),  #used to be 1024, 512
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256, 20)#usded to be 512,20
        )

    def forward(self, x):
        if torch.cuda.is_available():
           x = x.to(device="cuda")
        # conv layers
        x = self.conv_layer(x)
        # print(x.size(-4))
        # print(x.size(-3))
        # print(x.size(-2))
        # print(x.size(-1))
        # print(x.size(0))
        # print(x.size(1))
        # print(x.size(2))
        # print(x.size(3))
        # flatten
        x = x.view(x.size(0), -1)


        # fc layer
        x = self.fc_layer(x)

        return x

batch_size = 5
test_batch_size = 1
epochs = 20
lr = 1e-3
criterion = nn.CrossEntropyLoss()

#training
path = Path("/Users/games/Desktop/ML/path.pth")
model = CNN()
model.load_state_dict(torch.load(path))
model.eval()
optimizer = optim.Adam(model.parameters(), lr=lr)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=False)
losses = []
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    losses.append(test(model, val_loader))

torch.save(model.state_dict(), path)


7]]]])
2
tensor([[[[0.0549, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0588, 0.0118, 0.0039,  ..., 0.0980, 0.0745, 0.0667],
          [0.0000, 0.0000, 0.0000,  ..., 0.2902, 0.2863, 0.2745],
          ...,
          [0.0000, 0.0000, 0.0549,  ..., 0.4392, 0.4314, 0.4196],
          [0.0000, 0.0039, 0.0000,  ..., 0.4431, 0.4392, 0.4314],
          [0.0000, 0.0000, 0.0039,  ..., 0.4471, 0.4431, 0.4353]],

         [[0.0588, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0588, 0.0118, 0.0039,  ..., 0.0980, 0.0745, 0.0667],
          [0.0000, 0.0000, 0.0000,  ..., 0.2902, 0.2863, 0.2745],
          ...,
          [0.0000, 0.0000, 0.0549,  ..., 0.4392, 0.4314, 0.4196],
          [0.0000, 0.0039, 0.0000,  ..., 0.4431, 0.4392, 0.4314],
          [0.0000, 0.0000, 0.0039,  ..., 0.4471, 0.4392, 0.4353]],

         [[0.0549, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0588, 0.0118, 0.0000,  ..., 0.0980, 0.0745, 0.0667],
          [0.0000, 0.0000, 0.0000